In [ ]:
# Parameters
initial_return_thresh = 0.1
upstream = {
    "data_saving": {
        "data_train": "C:\\Users\\berkayg\\Desktop\\Coding env\\crypto-prediction-project\\products\\data\\raw_train_data.csv",
        "data_validation": "C:\\Users\\berkayg\\Desktop\\Coding env\\crypto-prediction-project\\products\\data\\raw_validation_data.csv",
    }
}
product = {
    "nb": "C:\\Users\\berkayg\\Desktop\\Coding env\\crypto-prediction-project\\products\\notebooks\\process_data.ipynb",
    "data_train": "C:\\Users\\berkayg\\Desktop\\Coding env\\crypto-prediction-project\\products\\data\\processed_train_data.csv",
    "data_validation": "C:\\Users\\berkayg\\Desktop\\Coding env\\crypto-prediction-project\\products\\data\\processed_validation_data.csv",
}


In [ ]:
import pandas as pd
import numpy as np

from scipy.signal import find_peaks

from datetime import datetime
import math
from sklearn.linear_model import LinearRegression
import plotly.express as px
from plotly import graph_objects as go

#import talib
import ta

In [ ]:
columns = ['Close', 'Volume', 'Price', 'trend_ema_fast', 'trend_ema_slow', 'trend_sma_fast', 'trend_sma_slow']

In [ ]:
def read_data(path):
    df = pd.read_csv(path, header=[0, 1],index_col=[0], parse_dates=[0])
    df = df.droplevel(0, axis=1)
    return df

In [ ]:
df_train = read_data(upstream["data_saving"]["data_train"])
df_validation = read_data(upstream["data_saving"]["data_validation"])
df_total = pd.concat([df_validation, df_train])

# Final version

In [ ]:
def _action(x):
    if not pd.isnull(x["loc_min"]):
        return "buy"
    elif not pd.isnull(x["loc_max"]):
        return "sell"
    else:
        return "neutral"

In [ ]:
def find_extremas(dataframe, **kwargs):
    """Finds local maximas/minimas in the series
    Returns: List of extremas indices and dataframe with extremas added
    """
    df = dataframe.copy()
    df["returns"] = df["Close"].pct_change().fillna(0) * 100
    df["returns"] = df["returns"].shift(-1)
    df = df.iloc[:-1]
    
    df['loc_max'] = df.iloc[find_peaks(df.reset_index(drop=True)['Close'].to_numpy(), **kwargs)[0]]['Close']
    df['loc_min'] = df.iloc[find_peaks(df.reset_index(drop=True)['Close'].to_numpy() * -1, **kwargs)[0]]['Close']
    
    idx_with_mins = np.where(df['loc_min'] > 0)[0]
    idx_with_maxs = np.where(df['loc_max'] > 0)[0]
    idx_concat = np.concatenate([idx_with_mins, idx_with_maxs])
    
    first_idx_bool = idx_with_mins.min() > idx_with_maxs.min()
    if first_idx_bool and 0 not in idx_concat:
        idx_with_mins = np.append(0, idx_with_mins)
        df.iloc[0, df.columns.get_loc("loc_min")] = df.iloc[0, df.columns.get_loc("Close")]
    
    elif not first_idx_bool and 0 not in idx_concat:
        idx_with_maxs = np.append(0, idx_with_maxs)
        df.iloc[0, df.columns.get_loc("loc_max")] = df.iloc[0, df.columns.get_loc("Close")]
    
    return idx_with_maxs, idx_with_mins, idx_concat, df

In [ ]:
def optimize_extremas(df, idx_concat,thresh=1):
    """Remove unsignificant peaks and dips
    Returns: Dataframe
    """
    df["idx"] = ((df["loc_min"].isnull()) & (df["loc_max"].isnull())).cumsum()
    df["idx"] = np.arange(0, df.shape[0])
    df["count"] = np.arange(0, df.shape[0])
    df["count"] = (df["count"].isin(idx_concat)).cumsum()
    df['cum_return'] = df.sort_index(ascending=False).groupby('count')['returns'].transform(np.cumsum)#.shift()
    df.loc[(df["loc_min"].isnull() == False) & (abs(df["cum_return"]) < thresh), "loc_min"] = np.nan
    df.loc[(df["loc_max"].isnull() == False) & (abs(df["cum_return"]) < thresh), "loc_max"] = np.nan
    df["action"] = df.apply(_action, axis=1)
    return df

In [ ]:
def plot_series(df):
    hover_data = {"Close": True, "Date": True, "returns":True, "cum_return":True, "idx":True, "count":True}
    fig_obj = px.line(x="Date", y="Close", data_frame=df.reset_index(), hover_data=hover_data)

    extrema_min = px.scatter(x="Date", y="loc_min", data_frame=df.reset_index(), hover_data=hover_data)
    extrema_min.update_traces(marker=dict(color='green'))

    extrema_max = px.scatter(x="Date", y="loc_max", data_frame=df.reset_index(), hover_data=hover_data)
    extrema_max.update_traces(marker=dict(color='red'))

    fig = fig_obj.data + extrema_min.data + extrema_max.data
    return go.Figure(fig)

In [ ]:
def prepare_data(df, symbol, prominence=0.8, thresh=0.5):
    print(f"CONFIG: Symbol: {symbol} - Prominence: {prominence} - Thresh: {thresh}")
    try:
        df = df[[symbol]].droplevel(axis=1, level=0)
    except:
        pass
    _, _, idx_concat, df = find_extremas(df, prominence=prominence)
    df = optimize_extremas(df, idx_concat, thresh=thresh)
    fig = plot_series(df)
    print(df.action.value_counts())
    return idx_concat, df[["Open", "High", "Low", "Close", "Volume", "action"]], fig
    

In [ ]:
def linear_regression(x, y):
    """
    performs linear regression given x and y. outputs regression coefficient
    """
    #fit linear regression
    lr = LinearRegression()
    lr.fit(x, y)
    
    return lr.coef_[0][0]

def n_day_regression(n, df, idxs):
    """
    n day regression.
    """
    #variable
    _varname_ = f'{n}_reg'
    df[_varname_] = np.nan

    for idx in idxs:
        if idx > n:
            
            y = df['Close'][idx - n: idx].to_numpy()
            x = np.arange(0, n)
            #reshape
            y = y.reshape(y.shape[0], 1)
            x = x.reshape(x.shape[0], 1)
            #calculate regression coefficient 
            coef = linear_regression(x, y)
            df.iloc[idx, df.columns.get_loc(_varname_)] = coef #add the new value
            
    return df


In [ ]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    center = df['center']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    # final_upperband = upperband = hl2 + (multiplier * atr)
    # final_lowerband = lowerband = hl2 - (multiplier * atr)

    final_upperband = upperband = center + (multiplier * atr)
    final_lowerband = lowerband = center - (multiplier * atr)

    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)

In [ ]:
def supertrend_processing(df):
    if "action" in df.columns:
        df = df.drop(columns=["action"])
    

    df["ph_arg"] = df["High"].rolling(2).apply(lambda x: np.argmax(x))
    df["ph_arg_rev"] = df["High"].sort_index(ascending=False).rolling(2).apply(lambda x: np.argmax(x))
    df["ph"] = (df["ph_arg"] == 1) & (df["ph_arg_rev"] == 1)
    df["ph"] = df.apply(lambda x: x["High"] if x["ph"] == 1 else np.nan, axis=1)

    df["pl_arg"] = df["Low"].rolling(2).apply(lambda x: np.argmin(x))
    df["pl_arg_rev"] = df["Low"].sort_index(ascending=False).rolling(2).apply(lambda x: np.argmin(x))
    df["pl"] = (df["pl_arg"] == 1) & (df["pl_arg_rev"] == 1)
    df["pl"] = df.apply(lambda x: x["Low"] if x["pl"] == 1 else np.nan, axis=1)

    df.drop(columns=["ph_arg_rev", "ph_arg", "pl_arg_rev", "pl_arg"], inplace=True)
    df = extract_pivot_centers(df)
    df["ph"] = np.where(df["ph"].isnull(), 0, 1)
    df["pl"] = np.where(df["pl"].isnull(), 0, 1)

    atr_period = 3
    high = df['High']
    low = df['Low']
    close = df['Close']
    center = df['center']
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    df["atr"] = atr
    df.dropna(inplace=True)
    
    return df

In [ ]:
def extract_pivot_centers(df):
    df["center"] = np.nan
    center = np.nan
    for i, k in enumerate(df.iterrows()):
        ph = k[1]["ph"]
        pl = k[1]["pl"]
        if not pd.isnull(ph):
            lastpp = ph
            center = lastpp
        elif not pd.isnull(pl):
            lastpp = pl
            center = lastpp
        else:
            k[1]["center"] = center
            continue

        if pd.isnull(center):
            center = (center * 2 + lastpp) / 3

        k[1]["center"] = center

    return df

In [ ]:
def process_data(df_source):
    df = ta.add_all_ta_features(df_source.drop(columns=["action"]), open="Open", high="High", low="Low", close="Close", volume="Volume")
    
    epsilon = 10e-10
    high = df["High"] - df["Low"]
    close = df["Close"] - df["Low"]
    df["Price"] = close/(high + epsilon)

    
    df = df[columns]
    df = n_day_regression(5, df, np.arange(0, df.shape[0]))
    df = n_day_regression(10, df, np.arange(0, df.shape[0]))
    df = n_day_regression(50, df, np.arange(0, df.shape[0]))

    df['action'] = df_source['action'].map(lambda x: 0 if x=="neutral" else 1)
    #df_processed['action'] = df_symbol['action'].map(action_dictionary)
    df["ema"] = df["trend_ema_fast"] / df["trend_ema_slow"]
    df["sma"] = df["trend_sma_fast"] / df["trend_sma_slow"]


    # import talib
    # df["RSI"] = talib.RSI(df_source["Close"])
    df["Stock_RSI"] = ta.momentum.StochRSIIndicator(df["Close"], window=5).stochrsi()
    df = df.dropna().drop(columns=["trend_ema_fast", "trend_ema_slow", "trend_sma_slow", "trend_sma_fast"])
    return df

In [ ]:
df_supertrend = supertrend_processing(df_total)

In [ ]:
custom_sum = lambda x: max(0, x.sum() - 1)
look_back_pivots = df_supertrend.rolling(5).agg({"ph": custom_sum, "pl": custom_sum}).add_prefix('5_step_lookback_').fillna(0)
df_supertrend = pd.concat([df_supertrend, look_back_pivots], axis=1)

In [ ]:
idx_concat, df_symbol, fig_symbol = prepare_data(df_train, "VIDTUSDT", prominence=None, thresh=initial_return_thresh)

In [ ]:
idx_concat_valid, df_symbol_valid, fig_symbol_valid = prepare_data(df_validation, "VIDTUSDT", prominence=None, thresh=initial_return_thresh)

In [ ]:
action_dictionary = {
    "buy": 1,
    "neutral": 0,
    "sell": 2
}

In [ ]:
df_processed_valid = process_data(df_symbol_valid)
df_processed_valid.head()

In [ ]:
df_processed = process_data(df_symbol)
df_processed.head()

In [ ]:
df_processed = pd.concat([df_processed, df_supertrend[["ph", "pl", "center", "5_step_lookback_ph", "5_step_lookback_pl", "atr"]]], axis=1).dropna()

In [ ]:
df_processed.head()

In [ ]:
df_processed_valid = pd.concat([df_processed_valid, df_supertrend[["ph", "pl", "center", "5_step_lookback_ph", "5_step_lookback_pl", "atr"]]], axis=1).dropna()

In [ ]:
df_processed_valid.head()

In [ ]:
df_processed.to_csv(product["data_train"])
df_processed_valid.to_csv(product["data_validation"])